In [1]:
pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.4/328.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00


In [2]:
!pip install openai scikit-learn

In [3]:
pip install transformers

In [4]:
pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
import json
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
from openai import OpenAI

def load_qa_data(filenames):
    qa_data = []
    for filename in filenames:
        with open(filename, 'r', encoding='utf-8') as f:
          qa_data.extend(json.load(f))
    return qa_data

filenames = ['/content/sample_data/qa_dataset.json','/content/sample_data/qa_dataset_mili.json', '/content/sample_data/qa_dataset_p.json']
qa_data = load_qa_data(filenames)

questions = [item['question'] for item in qa_data]
answers = [item['answer'] for item in qa_data]

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

question_embeddings = [get_bert_embeddings(question) for question in questions]

def find_best_answer(user_question):
    user_embedding = get_bert_embeddings(user_question)
    similarities = [cosine_similarity(user_embedding, q_emb).flatten()[0] for q_emb in question_embeddings]
    best_match_index = similarities.index(max(similarities))
    return answers[best_match_index]

client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

history = [
    {"role": "system", "content": "You are an intelligent assistant. You always provide well-reasoned answers that are both correct and helpful."},
    {"role": "user", "content": "Hello, introduce yourself to someone opening this program for the first time. Be concise."},
]

while True:
    # 사용자 입력
    user_input = input("> ")

    # 가장 적합한 답변 찾기
    best_answer = find_best_answer(user_input)

    # 히스토리 업데이트
    history.append({"role": "user", "content": user_input})
    history.append({"role": "assistant", "content": best_answer})

    # 응답 출력
    print(best_answer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

> 축제 언제 해?
학생예비군 훈련은 상반기 하반기 총 2번 합니다. 자세한 일정은 동의대학교 예비군 연대 창의관 213호, 890-1760~2로 연락해 자세한 일정을 참고하시기 바랍니다.
> 축제가 진행되는 장소는 어디야?
동의대학교 효민야구장입니다.
> 축제 날짜 알려줘
효민인 입장 대기줄은 효민 체육관 방면, 외부인 입장 대기줄은 효민 축구장 방면으로 대기하시면 됩니다.
> 예비군 어디서 해?
2024년 기준 영도구 예비군 훈련장에서합니다. 다만 이는 변경될 수 있습니다. 자세한 일정은 동의대학교 예비군 연대(창의관 213호, 890-1760~2)로 연락해 자세한 일정을 참고하시기 바랍니다.
